In [3]:
import sys

sys.path.insert(0, '..')

from ingestion import *

## Deputados


In [ ]:
import os

# Ingestão full

# Deputados
if os.path.exists('../data/raw/deputados.parquet'):
    print("O arquivo 'deputados.parquet' já existe. Ingestão não será realizada.")
else:
    df_deputados = obter_deputados()
    df_deputados.to_parquet('deputados.parquet', index=False)



if os.path.exists(f'../data/raw/detalhes_deputados.parquet'):
      print(f"O arquivo 'detalhes_deputados/.parquet' já existe. Ingestão não será realizada.")        
            
else:
    deputados = []
    # Detalhes dos deputados
    ids = df_deputados['id'].tolist()
    for id in ids:
        detalhe = obter_detalhe_deputado(id)
        print(f"Processando deputado {id} - Nome: {detalhe['dados']['ultimoStatus']['nome']}")
        if detalhe:
            deputados.append(detalhe['dados'])
    df_detalhes = pd.DataFrame(deputados)
    df_detalhes.to_parquet('detalhes_deputados.parquet', index=False)




O arquivo 'deputados.parquet' já existe. Ingestão não será realizada.
O arquivo 'detalhes_deputados/.parquet' já existe. Ingestão não será realizada.


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Função para gerar intervalos mensais
def gerar_intervalos_mensais(data_inicio, data_fim):
    data_inicio = datetime.strptime(data_inicio, '%Y-%m-%d')
    data_fim = datetime.strptime(data_fim, '%Y-%m-%d')
    intervalos = []
    while data_inicio < data_fim:
        proximo_mes = data_inicio + timedelta(days=32)
        proximo_mes = proximo_mes.replace(day=1)
        intervalos.append((data_inicio.strftime('%Y-%m-%d'), min(proximo_mes - timedelta(days=1), data_fim).strftime('%Y-%m-%d')))
        data_inicio = proximo_mes
    return intervalos

# Gerar intervalos mensais
intervalos = gerar_intervalos_mensais('2023-01-01', '2025-04-01')

# Ingestão de votações
todas_votacoes = pd.DataFrame()  # DataFrame vazio para armazenar todas as votações

for inicio, fim in intervalos:
    print(f"Ingestão de votações de {inicio} até {fim}")
    votacoes_mes = obter_votacoes(dataInicio=inicio, dataFim=fim)
    todas_votacoes = pd.concat([todas_votacoes, votacoes_mes], ignore_index=True)

# Concatenar todos os DataFrames
df_todas_votacoes = pd.concat(todas_votacoes, ignore_index=True)

# Salvar em arquivo Parquet
df_todas_votacoes.to_parquet('../data/rawtodas_votacoes.parquet', index=False)
print("Ingestão concluída e arquivo salvo como 'todas_votacoes.parquet'.")

In [12]:
votacoes = pd.DataFrame()

for df in todas_votacoes:
    votacoes = pd.concat([votacoes, df], ignore_index=True)

In [ ]:
votacoes.to_parquet('../data/raw/votacoes.parquet', index=False)



In [36]:
pd.set_option('colwidth', 1000)


ids=votacoes.query('descricao.str.contains("Sim", case=False)', engine='python')['id']

# votacoes[votacoes['id'].isin(sample)]['descricao']

In [37]:
votacoes = pd.read_parquet('../data/raw/votacoes.parquet')
from ingestion import obter_votos_votacao
# ids = votacoes['id'].tolist()
detalhes_votacoes = pd.DataFrame()
for id in ids:
    detalhe = obter_votos_votacao(id)

    if detalhe.empty:
        pass
    else:
        detalhe['idVotacao'] = id
        detalhes_votacoes = pd.concat([detalhes_votacoes, detalhe], ignore_index=True)
        print(f"Processando votação {id}")


    


Processando votação 2345468-38
Processando votação 2345378-38
Processando votação 2345368-56
Processando votação 2345368-50
Processando votação 2138680-43
Processando votação 2355363-12
Processando votação 2345484-83
Processando votação 2345484-39
Processando votação 2357625-5
Processando votação 2262108-41
Processando votação 2267419-90
Processando votação 2344938-71
Processando votação 2344938-60
Processando votação 2344938-48
Processando votação 2310837-8
Processando votação 2337246-43
Processando votação 2210127-55
Processando votação 2277651-56
Processando votação 344428-84
Processando votação 2369594-8
Processando votação 2363417-7
Processando votação 2363417-4
Processando votação 2363415-6
Processando votação 2367064-82
Processando votação 2367064-68
Processando votação 2367064-66
Processando votação 2360503-74
Processando votação 2360503-45
Processando votação 2360503-35
Processando votação 2360503-27
Processando votação 2196833-395
Processando votação 2196833-392
Processando v

In [39]:
detalhes_votacoes.to_parquet('../data/raw/votos.parquet', index=False)